# Building Recommendation Engines with PySpark

This course will show you how to build recommendation engines using `Alternating Least Squares` in PySpark. Using the popular MovieLens dataset and the Million Songs dataset, this course will take you step by step through the intuition of the Alternating Least Squares algorithm as well as the code to train, test and implement ALS models on various types of customer data.

## Table of Contents

- [Introduction](#intro)
- [How does ALS work?](#als)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

path = "data/dc36/"

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")
print(sc)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('First App').getOrCreate()

In [ ]:
# Return spark version
print(spark.version)

# Return python version
import sys
print(sys.version_info)

---
<a id='intro'></a>

## Why learn how to build recommendation engines?

<img src="images/spark6_001.png" alt="" style="width: 800px;"/>

<img src="images/spark6_002.png" alt="" style="width: 800px;"/>

<img src="images/spark6_003.png" alt="" style="width: 800px;"/>

## See the power of a recommendation engine

Taylor and Jane both like watching movies. Taylor only likes dramas, comedies, and romances. Jane likes only action, adventure, and otherwise exciting films. One of the greatest benefits of `ALS-based recommendation engines` is that they can identify movies or items that users will like, even if they themselves think that they might not like them. Take a look at the movie ratings that Taylor and Jane have provided below. It would stand to reason that their different preferences would generate different recommendations.

- Take a look at TJ_ratings using the .show() method and any other methods you prefer to see how each of them rated the various movies they've seen.
- Input user names into the get_ALS_recs() function provided to see what movies ALS recommends for Jane and Taylor based on the ratings provided. Do the ratings make sense to you?

```
In [1]: TJ_ratings.show()
+---------+--------------------+------+
|user_name|          movie_name|rating|
+---------+--------------------+------+
|   Taylor|            Twilight|   4.9|
|   Taylor|  A Walk to Remember|   4.5|
|   Taylor|        The Notebook|   5.0|
|   Taylor|Raiders of the Lo...|   1.2|
|   Taylor|      The Terminator|   1.0|
|   Taylor|      Mrs. Doubtfire|   1.0|
|     Jane|            Iron Man|   4.8|
|     Jane|Raiders of the Lo...|   4.9|
|     Jane|      The Terminator|   4.6|
|     Jane|           Anchorman|   1.2|
|     Jane|        Pretty Woman|   1.0|
|     Jane|           Toy Story|   1.2|
+---------+--------------------+------+

In [2]: get_ALS_recs(["Taylor","Jane"])
    userId  pred_rating                 title          genres
0   Taylor         3.89   Seven Pounds (2008)           Drama
1   Taylor         3.61      Cure, The (1995)           Drama
2   Taylor         3.55  Kiss Me, Guido (1997          Comedy
3   Taylor         3.29  You've Got Mail (199  Comedy|Romance
4   Taylor         3.27  10 Things I Hate Abo  Comedy|Romance
5   Taylor         3.26  Corrina, Corrina (19  Comedy|Drama|R
6     Jane         4.96           Fear (1996)        Thriller
7     Jane         4.85  Lord of the Rings: T  Adventure|Fant
8     Jane         4.70  Lord of the Rings: T  Adventure|Fant
9     Jane         4.55  No Holds Barred (198          Action
10    Jane         4.54  Lord of the Rings: T  Action|Adventu
11    Jane         4.30  Band of Brothers (20  Action|Drama|W
12    Jane         4.26   Transformers (2007)  Action|Sci-Fi|
```

## Recommendation Engine Types and Data Types

<img src="images/spark6_004.png" alt="" style="width: 800px;"/>

<img src="images/spark6_005.png" alt="" style="width: 800px;"/>

## Collaborative vs Content-Based Filtering Part II

Look at the df dataframe using the .show() method and/or the .columns method, and determine whether it is best suited for "collaborative filtering", "content-based filtering", or "both".

```
In [1]: df.show()
+------+-------+-----------------+--------+--------+-------------+------+
|UserId|MovieId|      Movie_Title|   Genre|Language|Year_Produced|rating|
+------+-------+-----------------+--------+--------+-------------+------+
| User1|   2112|     Finding Nemo|Animated| English|         2003|     3|
| User1|   2113|   The Terminator|  Action| English|         1984|     0|
| User1|   2114|       Spinal Tap|  Satire| English|         1984|     4|
| User1|   2115|Life Is Beautiful|   Drama| Italian|         1998|     4|
| User2|   2112|     Finding Nemo|Animated| English|         2003|     4|
| User2|   2113|   The Terminator|  Action| English|         1984|     0|
| User2|   2114|       Spinal Tap|  Satire| English|         1984|     0|
| User2|   2115|Life Is Beautiful|   Drama| Italian|         1998|     4|
| User3|   2112|     Finding Nemo|Animated| English|         2003|     1|
| User3|   2113|   The Terminator|  Action| English|         1984|     2|
| User3|   2114|       Spinal Tap|  Satire| English|         1984|     1|
| User3|   2115|Life Is Beautiful|   Drama| Italian|         1998|     0|
| User4|   2112|     Finding Nemo|Animated| English|         2003|     3|
| User4|   2113|   The Terminator|  Action| English|         1984|     1|
| User4|   2114|       Spinal Tap|  Satire| English|         1984|     0|
| User4|   2115|Life Is Beautiful|   Drama| Italian|         1998|     0|
+------+-------+-----------------+--------+--------+-------------+------+
```

Possible Answers
- Collaborative filtering
- Content-based filtering
- Both collaborative and content-based filtering (correct)

Because this dataset includes descriptive tags like genre and language, as well as user ratings, it is suited for both collaborative and content-based filtering.

## Implicit vs Explicit Data

Recall the differences between implicit and explicit ratings. Take a look at the df1 dataframe to understand whether the data includes implicit or explicit ratings data.

- Use the .columns and .show() methods to get an idea of the data provided, and see if the data includes implicit or explicit ratings.
- Type "implicit" or "explicit" based on whether you think this data contains "implicit" ratings or "explicit" ratings. Name your response answer.

```
In [1]: df1.columns
Out[1]: ['Movie_Title', 'Genre', 'Num_Views']

In [2]: df1.show()
+--------------------+------------------+---------+
|         Movie_Title|             Genre|Num_Views|
+--------------------+------------------+---------+
|        Finding Nemo|Animated Childrens|       12|
|           Toy Story|Animated Childrens|        6|
|            Iron Man|            Action|        1|
|     Captain America|            Action|        1|
|     The Incredibles|Animated Childrens|        9|
|              Frozen|Animated Childrens|       22|
|The Shawshank Red...|             Drama|        2|
|  Rabbit Proof Fence|             Drama|        2|
|Searching for Sug...|       Documentary|        3|
|              Powder|             Drama|        1|
|        The Fugitive|            Action|        2|
+--------------------+------------------+---------+
```
This dataset includes user behavior counts which are used as implicit ratings.

## Ratings data types

Markus watches a lot of movies, including documentaries, superhero movies, classics, and dramas. Drawing on your previous experience with Spark, use the markus_ratings dataframe, which contains data on the number of times Markus has seen movies in various genres, and think about whether these are implicit or explicit ratings. Use the groupBy() method to determine which genre has the highest rating, which could likely influence what recommendations ALS would generate for Markus.

- Use the groupBy() method to group the markus_ratings dataframe by "Genre".
- Apply the .sum() method to get the total number of movies watched for each genre.
- Be sure to add the .show() method at the end to view the counts.

```
In [1]: markus_ratings.show()
+--------------------+------------------+---------+
|         Movie_Title|             Genre|Num_Views|
+--------------------+------------------+---------+
|        Finding Nemo|Animated Childrens|       12|
|           Toy Story|Animated Childrens|        6|
|            Iron Man|            Action|        1|
|     Captain America|            Action|        1|
|     The Incredibles|Animated Childrens|        9|
|              Frozen|Animated Childrens|       22|
|The Shawshank Red...|             Drama|        2|
|  Rabbit Proof Fence|             Drama|        2|
|Searching for Sug...|       Documentary|        3|
|              Powder|             Drama|        1|
|        The Fugitive|            Action|        2|
+--------------------+------------------+---------+

In [2]: # Group the data by "Genre"
        markus_ratings.groupBy("Genre").sum().show()
+------------------+--------------+
|             Genre|sum(Num_Views)|
+------------------+--------------+
|             Drama|             5|
|       Documentary|             3|
|            Action|             4|
|Animated Childrens|            49|
+------------------+--------------+
```
Markus seems to like animated children's movies. Or perhaps his 3 kids use his movie streaming account more than he does.

## Uses for Recommendation Engines

<img src="images/spark6_006.png" alt="" style="width: 800px;"/>

<img src="images/spark6_007.png" alt="" style="width: 800px;"/>

<img src="images/spark6_008.png" alt="" style="width: 800px;"/>

<img src="images/spark6_009.png" alt="" style="width: 800px;"/>

`Latent features` aren't directly observable by humans, and need mathematical operations to uncover them.

## Confirm understanding of latent features

Matrix P is provided here. Its columns represent movies and its rows represent several latent features. Use your understanding of Spark commands to view matrix P and see if you can determine what some of the latent features might represent. After examining the matrix, look at the dataframe Pi, which contains a rough approximation of what these latent features could represent. See if you weren't far off.

- Examine matrix P using the .show() method.
- Examine matrix Pi using the .show() method.

```
<script.py> output:
    +--------+------------+--------+---------+------------+------+----------+
    |Iron Man|Finding Nemo|Avengers|Toy Story|Forrest Gump|Wall-E|Green Mile|
    +--------+------------+--------+---------+------------+------+----------+
    |     0.2|         2.4|     0.1|      2.4|           0|   2.5|         0|
    |     1.5|         1.4|     1.4|      1.3|         1.8|   1.8|       2.5|
    |     2.5|         1.1|     2.4|      0.9|         0.2|   0.9|      0.09|
    |     1.9|           2|     1.5|      2.2|         1.2|   0.3|      0.01|
    |       0|           0|       0|      2.3|         2.2|     0|       2.5|
    +--------+------------+--------+---------+------------+------+----------+
    
    +---------+--------+------------+--------+---------+------------+------+----------+
    | Lat Feat|Iron Man|Finding Nemo|Avengers|Toy Story|Forrest Gump|Wall-E|Green Mile|
    +---------+--------+------------+--------+---------+------------+------+----------+
    | Animated|     0.2|         2.4|     0.1|      2.4|           0|   2.5|         0|
    |    Drama|     1.5|         1.4|     1.4|      1.3|         1.8|   1.8|       2.5|
    |Superhero|     2.5|         1.1|     2.4|      0.9|         0.2|   0.9|      0.09|
    |   Comedy|     1.9|           2|     1.5|      2.2|         1.2|   0.3|      0.01|
    |Tom Hanks|       0|           0|       0|      1.8|         2.2|     0|       2.5|
    +---------+--------+------------+--------+---------+------------+------+----------+
```

---
<a id='als'></a>

## How does ALS work?

<img src="images/spark6_010.png" alt="" style="width: 800px;"/>

<img src="images/spark6_011.png" alt="" style="width: 800px;"/>

<img src="images/spark6_012.png" alt="" style="width: 800px;"/>

<img src="images/spark6_013.png" alt="" style="width: 800px;"/>

<img src="images/spark6_014.png" alt="" style="width: 800px;"/>

<img src="images/spark6_015.png" alt="" style="width: 800px;"/>

<img src="images/spark6_016.png" alt="" style="width: 800px;"/>

<img src="images/spark6_017.png" alt="" style="width: 800px;"/>

<img src="images/spark6_018.png" alt="" style="width: 800px;"/>

<img src="images/spark6_019.png" alt="" style="width: 800px;"/>

## 

In [ ]:
# Terminate the cluster
spark.stop()

In [ ]:
<img src="images/spark6_020.png" alt="" style="width: 800px;"/>

In [ ]:
---
<a id='intro'></a>